<img src="../img/crowdlearning-etic.png" alt="Logo ETIC" align="right">


<h1><font color="#004D7F" size=6>Pipelines</font></h1>

<br>
<br>
<br>
<br>
<div style="text-align: right">
<font color="#004D7F" size=3>Antonio Jesús Gil</font><br>
<font color="#004D7F" size=3>Fundamentos de Machine Learning</font><br>

</div>

Un Pipeline consiste en una combinación de estimadores que se ejecutan como si fuesen uno. Más concretamente se trata de una secuencia de transformaciones y el último es un estimador de cualquier tipo (transformador, clasificador, etc.).

Pipeline se encarga de ir llamando a las funciones `fit` y `transform` de cada uno de los transformadores hasta que llega al último de ellos, siendo la entrada de la función `fit` el resultado del `transform` anterior.

Pipeline tendrá aquellas funciones correspondientes a su último estimador, es decir, si al final hay un clasificador, el Pipeline tendrá las funciones `fit`, `predict` y `score`, si es un transformador, `fit` y `transform`.

Se trata de una herramienta muy útil que permite reducir el tamaño del código y ayuda a la reproducibilidad de diferentes experimentos.

En el primer ejemplo vamos a definir un __pipeline__ con algunas de las clases que hemos utilizado. Para ello vamos a utilizar la base de datos Wisconsin en formato csv vista en el apartado Transformaciones. Las características en `wisconsin_data`, dispone de ciertos valores perdidos, por lo que __no podía ser pasado__ directamente al clasificador. Por eso, vamos a crear un __pipeline__ que primero utilice un `SimpleImputer` y luego llame a `LogisticRegression`.

Igual que en todos los casos anteriores, utilizaremos el __pipeline__ con las funciones `fit`, `predict` y `score`, de la misma forma que si se tratase de un clasificador. Para crear un Pipeline se utiliza una lista de tuplas (clave, valor) donde la clave es un string representativo y el valor es el estimador.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer

import pandas as pd
wisconsin = pd.read_csv('wisconsin.csv')
wisconsin.head(5)

In [ ]:
# Lo primero es separar los atributos de la clase.
wisconsin_data = wisconsin.drop('label',1)
wisconsin_target = wisconsin['label']

In [ ]:
wisconsin_data.isnull().values.any()

In [ ]:
# creamos un objeto pipeline compuesto de dos objetos estimador: imputer y logisticregression
pipe = Pipeline([('preprocesado', SimpleImputer()), ('clasificador', LogisticRegression())])

# hacemos fit con wisconsin_data y wisconsin_target
pipe.fit(wisconsin_data,wisconsin_target)

# obtenemos la tasa de acierto con la función score del pipeline
print("Tasa de acierto = {0:.2%}".format(pipe.score(wisconsin_data,wisconsin_target)))

<a id="section42"></a> 
### <font color="#004D7F">Propiedades del Pipeline</font>

Es posible acceder a los elementos que componen el Pipeline de dos formas, mediante una lista con `steps`:

In [ ]:
pipe.steps

O mediante un diccionario con `named_steps`:

In [ ]:
pipe.named_steps

Es posible también acceder a los hiperparametros de los estimadores ya definidos en el pipeline mediante `set_params` indicando en la función el parametro que se desea modificar de la siguiente forma `<clave del estimador>__<hiperparametro>`. Veamos un ejemplo de esto:

In [ ]:
# Actualmente la estrategia para rellenar los nan's es 'mean'
pipe.set_params(preprocesado__strategy='median')
# Y ahora la strategy es la mediana
pipe.named_steps

Pero no sólo eso, podemos cambiar también un estimador entero gracias a esta función. Esto es especialmente útil cuando tenemos que probar una serie de clasificadores y el preprocesamiento es siempre el mismo.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

pipe.set_params(clasificador=KNeighborsClassifier())
pipe.named_steps

Finalmente, podemos obtener los parametros aprendidos en las diferentes etapas del Pipeline si accedemos directamente al estimador con `steps` o `named_steps`. Si usamos el Pipeline actual y realizamos `fit`, podemos ver que es lo que aprende la regresión logística a través de los coeficientes (`coef_`):

In [ ]:
pipe = Pipeline([('preprocesado', SimpleImputer()), ('clasificador', LogisticRegression())])

pipe.fit(wisconsin_data,wisconsin_target)
print(pipe.named_steps['clasificador'].coef_)

Esto representa los pesos que se han aprendido para las 10 variables.